In [4]:
# THIS DATASET GETS THE MP3 PREVIEWS FROM THE SONGS IN A DATASET, CONNECTING WITH SPOTIFY API
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import os
from tqdm import tqdm
import requests
import time

# Read client_id and client_secret from keys.txt
with open("keys.txt", "r") as keys_file:
    client_id = keys_file.readline().strip()
    client_secret = keys_file.readline().strip()

scope = "user-library-read app-remote-control playlist-read-private playlist-read-collaborative user-read-currently-playing streaming"
redirect_uri = 'http://localhost:8081/callback'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri))

# Read in the CSV file
df = pd.read_csv('../our_data/spotify_data_all_songs.csv')
df = df.drop(range(3595))

# Retrieve the current user's profile information
user = sp.current_user()
print(user)

# Iterate through each row in the DataFrame
for i, element in tqdm(df.iterrows()):
    # Get the Spotify ID for the song
    spotify_id = element['track_uri']
    #print(spotify_id)

    retry = 2
    while retry > 0:
        try:
            track_info = sp.track(track_id=spotify_id)
            # Get the preview URL for the track
            url = track_info['preview_url']
            #print(url)

            # Download the preview mp3 file
            if url is None:
                #print('url is None')
                break
            mp3_file = f"mp3-new-download-session/{spotify_id}.mp3"
            if os.path.isfile(mp3_file):
                #print(f"{mp3_file} already exists. Skipping...")
                break
            os.makedirs('mp3-new-download-session', exist_ok=True)
            response = requests.get(url)
            with open(mp3_file, "wb") as f:
                #print("Writing mp3")
                f.write(response.content)
            break
        except Exception as e:
            print(f"Error occurred while processing {spotify_id}: {e}. Retrying in 31 seconds.")
            time.sleep(31)
            retry -= 1
    time.sleep(0.15)

    if retry == 0:
        print(f"Failed to process {spotify_id} after 2 attempts. Waiting for 30 seconds and moving to the next song.")
        time.sleep(30)


{'display_name': 'Pascu CS', 'external_urls': {'spotify': 'https://open.spotify.com/user/31zq42jmm5zryt3q6rtgw4dej434'}, 'followers': {'href': None, 'total': 0}, 'href': 'https://api.spotify.com/v1/users/31zq42jmm5zryt3q6rtgw4dej434', 'id': '31zq42jmm5zryt3q6rtgw4dej434', 'images': [], 'type': 'user', 'uri': 'spotify:user:31zq42jmm5zryt3q6rtgw4dej434'}


1335it [06:28,  3.44it/s]


KeyboardInterrupt: 